# Neural Network

## Perceptrons

In [1]:
from linearalgebra import Vector, dot

In [2]:
def step_function(x: float) -> float:
    return 1.0 if x >= 0 else 0.0

In [3]:
def perceptron_output(weights: Vector, bias: float, x: Vector) -> float:
    ''' return 1 if the perceptron "fires",  0 if not'''
    calculation = dot(weights, x) + bias
    return step_function(calculation)

In [6]:
add_weights = [2., 2]
and_bias = -3.

In [7]:
perceptron_output(and_weights, and_bias, [1,1])

NameError: name 'and_weights' is not defined